In [1]:
from runtest import *
from Data_binary import *
from statistics import median
import numpy as np
import time

In [2]:
env = Data(unknown_rate=1)
env.loadfile("survey.csv") # change this to the test file
env.normalize()
env.alpha = 0
env.cluster_K_means(7)
# this makes it so when we do the ranking, we only check against this number
# of data points; ignore for now
#env.set_validation(2000)
# makes the costs uniform; we won't have groups either
env.set_costs()

# partition into training/test sets
test_env = env.split(0.80)
print("training points:", len(env.data))
print("test points:", len(test_env.data))

training points: 4657
test points: 1163


In [ ]:
costs = [(i+1)/10 for i in range(10)] # list of maximum budgets
gammas = {}
for cost in costs:
    gammas[cost]=0.8
results = [] # stores all the results for each value of budget
for c in costs:
    env.max_cost = c
    test_env.max_cost = c
    # take the average of 3 trainings
    r1 = []
    r2 = []
    
    print("cost",c)
    
    agent_distance = testAgent(env,test_env, "survey", c, gamma=gammas[c], max_eps=500, epsilon_decay=0.98)
    r1.append(agent_distance)
    
    for i in range(3):
        start = time.time()
        random_distance = testRandom(env, test_env, c)
        r2.append(random_distance)
        print("random:", random_distance, "took", time.time()-start,"seconds")
    print("median for RL:", median(r1), "median for random:",median(r2))
    print("mean for RL:", np.mean(r1), "mean for random:",np.mean(r2))
    print()
    results.append([r1,r2])

cost 0.1
random: 6541.354164365904 took 23.78960871696472 seconds
random: 6542.181361178117 took 23.751585721969604 seconds
random: 6489.180997453931 took 24.08489203453064 seconds
median for RL: 6671.775504352538 median for random: 6541.354164365904
mean for RL: 6671.775504352538 mean for random: 6524.238840999317

cost 0.2
random: 5563.270864834373 took 24.803722143173218 seconds
random: 5538.81258862772 took 24.61825203895569 seconds
random: 5533.461611012349 took 24.816554307937622 seconds
median for RL: 4671.4068348421115 median for random: 5538.81258862772
mean for RL: 4671.4068348421115 mean for random: 5545.181688158147

cost 0.3
random: 4965.429961504316 took 25.832474946975708 seconds


In [ ]:
import matplotlib.pyplot as plt

# this value is the total distance between p and 5 closest predicted points to p
resRL = [results[i][0] for i in range(len(results))]
resRAND = [median(results[i][1]) for i in range(len(results))]
plt.plot(costs, resRL, 'r', costs, resRAND)
plt.show()

In [ ]:
# we will write the results to BENCHMARK/<test>.csv
f = open('BENCHMARK/survey_test.csv', 'w')

def write(f,lst):
    for l in lst:
        f.write(str(l))
        f.write(' ')

# first write the number of test points
f.write(str(len(test_env.data)))
f.write('\n')
        
# write the cost
write(f,costs)
f.write('\n')

# write the RL result
write(f,resRL)
f.write('\n')

# finally write the random results
write(f, resRAND)
f.write('\n')

f.close()